In [69]:
#Importamos la librerías y variables necesarias

import mysql.connector
import pandas as pd
from flask import Flask, jsonify, request
import plotly as plt
import os

import pickle
import plotly.graph_objects as go
import plotly.express as px
import plotly

In [70]:
cnx = mysql.connector.connect(
    user="admin",
    password="admin123",
    host="test-db.cze2nnbbx5pc.eu-west-3.rds.amazonaws.com",
    database="prueba"
)
cursor = cnx.cursor()

#Definimos una función mediante la que ejecutar las querys devolviendo un dataframe
def make_query(code):
    cursor.execute(code)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]  # Obtener los nombres de las columnas
    df = pd.DataFrame(results, columns=column_names)  # Crear el DataFrame
    return df


In [65]:
df=make_query("SELECT months_left FROM replacement")

counts = df['months_left'].value_counts().sort_index()
counts_filtered = counts.loc[counts.index <= -50] # Filtro para 24 meses
counts_filtered
# fig = px.line(x=counts_filtered.index, y=counts_filtered.values, title="Prediction attrition for next 24 months") # Gráfica de series de tiempo

# fig.update_traces(line_width=3, mode='lines+markers', hovertemplate='Month: %{x}<br>Nº of attrition: %{y}')  

# fig.update_layout(xaxis=dict(
#     tickmode='array',
#     tickvals=counts_filtered.index,
#     ticktext=counts_filtered.index
# ), xaxis_title="next 24 months", 
# yaxis_title="Nº of attrition", 
# title_x=0.5, 
# title_font={'size': 24},
# width=840,
# height=420,
# paper_bgcolor='rgba(0,0,0,0)',
# font_family="Roboto",
# font_color="#1D3557",
# title_font_family="Roboto",
# title_font_color="#1D3557",
# legend_title_font_color="#1D3557")
# fig.show()

-138     1
-88      1
-71      4
-70      5
-69      7
-68      3
-67      6
-66      6
-65      4
-64      1
-63      8
-62      6
-61      5
-60      7
-59     13
-58     13
-57     19
-56     15
-55     16
-54     10
-53      9
-52     23
-51     15
-50      8
Name: months_left, dtype: int64

In [48]:
def get_graph_pie():

    df=make_query("SELECT months_left FROM replacement")

    # Supongamos que tienes un dataframe llamado "df" con una columna llamada "column_name"
    column_name = df['months_left'].astype(float)
    # Calcula los cuartiles utilizando la función quantile()
    q1 = column_name.quantile(0.25)
    q2 = column_name.quantile(0.5)
    q3 = column_name.quantile(0.75)
    # Cuenta el número de valores en cada cuartil
    count_q1 = column_name[column_name <= q1].count()
    count_q2 = column_name[(column_name > q1) & (column_name <= q2)].count()
    count_q3 = column_name[(column_name > q2) & (column_name <= q3)].count()
    count_q4 = column_name[column_name > q3].count()
    column1=["Very High","High","Medium","Low"]
    column2=[count_q1,count_q2,count_q3,count_q4]
    data = {"risk": column1, "data": column2}
    df = pd.DataFrame(data)
    df['percentage'] = (df['data'] / df['data'].sum()) * 100

    count_values = df.set_index('risk')['data']

    colors = {
        "Low": "#00CC96",
        "Medium": "#B6E880",
        "High": "#FFA15A",
        "Very High": "#EF553B"
    }
    fig = go.Figure(data=[go.Pie(labels=count_values.index, values=count_values.values, hole=0.4, pull=[0.05, 0.05, 0.05, 0.05])])
    fig.update_traces(
        marker=dict(colors=[colors[label] for label in count_values.index]),
        textfont=dict(size=22)  # Aumentar el tamaño de los valores
    )
    fig.update_layout(
        title={"text":"Distribution Risk Attrition",
            "x":0.5},
        title_font=dict(size=24),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            traceorder='normal'
        )
    )

    return fig.show()

In [49]:
get_graph_pie()

In [3]:
df = make_query("SELECT role, risk FROM replacement")
# percentage_df = df.groupby(['risk', 'role']).size().groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index(name='percentage')

In [51]:
df_agg = df.groupby(['role', 'risk']).size().reset_index(name='count')
df_agg['percentage'] = df_agg.groupby('role')['count'].apply(lambda x: (x / x.sum()) * 100)

fig = go.Figure()

colors = {
    "Low": "#00CC96",
    "Medium": "#B6E880",
    "High": "#FFA15A",
    "Very high": "#EF553B"
}

for risk in df_agg['risk'].unique():
    df_filtered = df_agg[df_agg['risk'] == risk]
    fig.add_trace(go.Bar(
        x=df_filtered['percentage'],
        y=df_filtered['role'],
        name=risk,
        orientation='h',
        marker=dict(color=colors[row['risk']]),
        text=f"{percentage:.2f}%",
        textposition='auto'
    ))

fig.update_layout(
    title={
        'text': 'Distribution risk attrition by job role',
        'font': {'size': 24}
    },
    xaxis=dict(title='Porcentaje'),
    yaxis=dict(title='JobRole'),
    barmode='stack',
    autosize=False,
    width=800,
    height=500,
    title_x=0.5,
)
fig.show()

NameError: name 'row' is not defined

In [84]:
df_agg = df.groupby(['role', 'risk']).size().reset_index(name='count')
df_agg['percentage'] = df_agg.groupby('role')['count'].apply(lambda x: (x / x.sum()) * 100)

fig = go.Figure()

colors = {
    "Low": "#00CC96",
    "Medium": "#B6E880",
    "High": "#FFA15A",
    "Very high": "#EF553B"
}

for risk in df_agg['risk'].unique():
    df_filtered = df_agg[df_agg['risk'] == risk]
    fig.add_trace(go.Bar(
        x=df_filtered['percentage'],
        y=df_filtered['role'],
        name=risk,
        orientation='h',
        marker=dict(color=colors[risk]),
        text=df_filtered['percentage'].apply(lambda x: f"{x:.2f}%"),
        textposition='auto'
    ))

fig.update_layout(
    title={
        'text': 'Distribution risk attrition by job role',
        'font': {'size': 24}
    },
    xaxis=dict(title='Porcentaje'),
    yaxis=dict(title='JobRole'),
    barmode='stack',
    autosize=False,
    width=800,
    height=500,
    title_x=0.5,
)
fig.show()


In [44]:
df = make_query("SELECT job_level, risk FROM replacement")
df_agg = df.groupby(['job_level', 'risk']).size().reset_index(name='count')
df_agg['percentage'] = df_agg.groupby('job_level')['count'].apply(lambda x: (x / x.sum()) * 100)

fig = go.Figure()

colors = {
    "Low": "#0F9D58",
            "Medium": '#FFFF00',
            "High": "#FABC09",
            "Very high": "#DB4437"
}

x_order = ["Low", "Medium", "High", "Very high"]
for risk in x_order:
    df_filtered = df_agg[df_agg['risk'] == risk]
    fig.add_trace(go.Bar(
        x=df_filtered['percentage'],
        y=df_filtered['job_level'],
        name=risk,
        orientation='h',
        marker=dict(color=colors[risk]),
        text=df_filtered['percentage'].apply(lambda x: f"{x:.2f}%"),
        textposition='auto'
    ))


legend_order = [2, 0, 1, 3]
fig.update_layout(
    title={
        'text': 'Distribution risk attrition by job level',
        'font': {'size': 24}
    },
    
    yaxis=dict(title='Job Level'),
    barmode='stack',
    autosize=False,
    width=800,
    height=500,
    title_x=0.5,
    # legend=dict(traceorder='normal',
    #             tracegroupgap=0,
    #             order=legend_order,
    # xaxis=dict(
    #     categoryorder='array',
    #     categoryarray=x_order  # Utilizar el orden especificado en el eje x
    # )
)
fig.show()


In [30]:
df = make_query("SELECT job_level, risk FROM replacement")
df_agg = df.groupby(['job_level', 'risk']).size().reset_index(name='count')
df_agg['percentage'] = df_agg.groupby('job_level')['count'].apply(lambda x: (x / x.sum()) * 100)

x_order = ["Low", "Medium", "High", "Very high"]

fig = go.Figure()

colors = {
    "Low": "#0F9D58",
            "Medium": '#FFFF00',
            "High": "#FABC09",
            "Very high": "#DB4437"
}

for risk in df_agg['risk'].unique():
    df_filtered = df_agg[df_agg['risk'] == risk]
    df_filtered['job_level'] = pd.Categorical(df_filtered['job_level'], categories=x_order, ordered=True)
    df_filtered = df_filtered.sort_values('job_level')
    fig.add_trace(go.Bar(
        x=df_filtered['percentage'],
        y=df_filtered['job_level'],
        name=risk,
        orientation='h',
        marker=dict(color=colors[risk]),
        text=df_filtered['percentage'].apply(lambda x: f"{x:.2f}%"),
        textposition='auto'
    ))



fig.update_layout(
    title={
        'text': 'Distribution risk attrition by job level',
        'font': {'size': 24}
    },
    
    yaxis=dict(title='Job Level'),
    barmode='stack',
    autosize=False,
    width=800,
    height=500,
    title_x=0.5,
    xaxis=dict(
        categoryorder='array',
        categoryarray=x_order  # Utilizar el orden especificado en el eje x
    )
)
fig.show()

c:\Users\gylmo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
import plotly.graph_objects as go
import pandas as pd

x_order = ["Low", "Medium", "High", "Very high"]  # Orden especificado

df = make_query("SELECT job_level, risk FROM replacement")
df_agg = df.groupby(['job_level', 'risk']).size().reset_index(name='count')
df_agg['percentage'] = df_agg.groupby('job_level')['count'].apply(lambda x: (x / x.sum()) * 100)

fig = go.Figure()

colors = {
    "Low": "#00CC96",
    "Medium": "#B6E880",
    "High": "#FFA15A",
    "Very high": "#EF553B"
}

for risk in df_agg['risk'].unique():
    df_filtered = df_agg[df_agg['risk'] == risk]
    df_filtered['job_level'] = pd.Categorical(df_filtered['job_level'], categories=x_order, ordered=True)  # Convertir a tipo "categorical" con el orden deseado
    df_filtered = df_filtered.sort_values('job_level')  # Ordenar el DataFrame según el orden deseado
    fig.add_trace(go.Bar(
        x=df_filtered['percentage'],
        y=df_filtered['job_level'],
        name=risk,
        orientation='h',
        marker=dict(color=colors[risk]),
        text=df_filtered['percentage'].apply(lambda x: f"{x:.2f}%"),
        textposition='auto'
    ))

fig.update_layout(
    title={
        'text': 'Distribution risk attrition by job level',
        'font': {'size': 24}
    },
    xaxis=dict(title='Percentage'),
    yaxis=dict(title='Job Level'),
    barmode='stack',
    autosize=False,
    width=800,
    height=500,
    title_x=0.5
)
fig.show()


c:\Users\gylmo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
# Obtener el conteo de valores para cada categoría en Prediction_nº_Months

df=make_query("SELECT months_left FROM replacement")
counts = df['months_left'].value_counts().sort_index()

# Filtrar los valores menores o iguales a 24
counts_filtered = counts.loc[counts.index <= 24]

# Crear la gráfica de series de tiempo
fig = px.line(x=counts_filtered.index, y=counts_filtered.values, title="Prediction attrition for next 24 months")

fig.update_traces(line_width=3, mode='lines+markers', hovertemplate='Month: %{x}<br>Nº of attrition: %{y}')  # Ajustar el grosor de la línea, agregar marcadores circulares y personalizar etiquetas

fig.update_layout(xaxis=dict(
    tickmode='array',
    tickvals=counts_filtered.index,
    ticktext=counts_filtered.index
), xaxis_title="next 24 months", yaxis_title="Nº of attrition", title_x=0.5, title_font={'size': 24})  # Centrar el título

# Mostrar la gráfica
fig.show()

In [41]:
df=make_query("SELECT risk FROM replacement")

id = 48

# Mapear categorías de riesgo a valores numéricos
risk_mapping = {'Low': 13, 'Medium': 38, 'High': 63, 'Very high':88}
df['risk_value'] = df['risk'].map(risk_mapping)

# Definir el layout del gráfico de gauge
layout = go.Layout(
    title='Gauge Chart',
)

# Crear la figura de gauge
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode='gauge',
    value=df['risk_value'].iloc[id],
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Risk"},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': 'green'},
           'threshold': {'line': {'color': 'red', 'width': 10}, 'thickness': .99, 'value': 75}
           }))

fig.update_layout(layout)

# Mostrar el gráfico de gauge
fig.show()

In [44]:
import plotly.graph_objects as go
import numpy as np
df=make_query("SELECT life_balance FROM replacement")
risk_mapping = {'Bad': 13, 'Good': 38, 'Better': 63, 'Best':88}
df['balance_value'] = df['life_balance'].map(risk_mapping)
id = 1
num_steps = 100  # Número de pasos para la escala continua
# Generar colores interpolados para los pasos de la escala continua
colors_interpolated = [f'rgb({int(255*(1-np.sqrt(i/num_steps)))}, {int(255*np.sqrt(i/num_steps))}, 0)' for i in range(num_steps)]
tick_labels = ['Bad', 'Good', 'Better', 'Best']
tick_values = [20, 40, 60, 80]
layout = go.Layout(
    title='Gauge Chart',
    paper_bgcolor='rgba(0, 0, 0, 0)'
)
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode='gauge',
    value=df['balance_value'].iloc[id],
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': "Work Life Balance"},
    gauge={
        'axis': {'range': [0, 100], 'tickmode': 'array', 'tickvals': tick_values,'ticktext':tick_labels},
        'bar': {'color': 'rgba(0, 0, 0, 0)', 'thickness': 0.75},
        'steps': [{'range': [i, i + 1], 'color': colors_interpolated[i]} for i in range(num_steps)],
        'threshold': {
            'line': {'color': 'black', 'width': 15},
            'thickness': .75,
            'value': df['balance_value'].iloc[id]
        },
        
    }
))
text_value = df['life_balance'].iloc[id]
fig.add_annotation(
    x=0.5, y=0.2,  # Coordenadas en el gráfico (0-1)
    text=text_value,
    showarrow=False,
    font=dict(size=60)
)
fig.update_layout(layout)
fig.show()

In [62]:
def make_query_json(code):
    cnx.close()
    cnx.connect()
    '''
    Función principal de la API que permite
    hacer una query a la BD y devuelve el DF
    resultante para hacer gráficas
    
    La query utiliza como motor MySQL y debe
    seguir la sintaxis de SQL
    '''
    cursor.execute(code)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description] 
    df = pd.DataFrame(results, columns=column_names)
    
    return df.iloc[0]

In [63]:
make_query_json('''SELECT COUNT(months_left) as total_filas
FROM prueba.replacement
WHERE months_left < 25
AND months_left > -1''')

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [72]:
import numpy as np
num_steps = 100 

df=make_query("SELECT life_balance FROM replacement")

# Mapeo para poder presentar el riesgo numéricamente

risk_mapping = {'Bad': 13, 'Good': 38, 'Better': 63, 'Best':88}
df['balance_value'] = df['life_balance'].map(risk_mapping)
id=15

# Generar colores interpolados para los pasos de la escala continua
colors_interpolated = [f'rgb({int(255*(1-np.sqrt(i/num_steps)))}, {int(255*np.sqrt(i/num_steps))}, 0)' for i in range(num_steps)]
tick_labels = ['Bad', 'Good', 'Better', 'Best']
tick_values = [20, 40, 60, 80]

# Tamaño de figura adaptado a la web
layout = go.Layout(
title='',
width=500,
height=300,
paper_bgcolor='rgba(0,0,0,0)', # Fondo transparente
font_family="Roboto",
font_color="#1D3557",
title_font_family="Roboto",
title_font_color="#1D3557",
legend_title_font_color="#1D3557"
)

fig = go.Figure()
fig.add_trace(go.Indicator(
    mode='gauge',
    value=df['balance_value'].iloc[id],
    domain={'x': [0, 1], 'y': [0, 1]},
    title={'text': ""},
    gauge={
        'axis': {'range': [0, 100], 'tickmode': 'array', 'tickvals': tick_values,'ticktext':tick_labels},
        'bar': {'color': 'rgba(0, 0, 0, 0)', 'thickness': 0.75},
        'steps': [{'range': [i, i + 1], 'color': colors_interpolated[i]} for i in range(num_steps)],
        'threshold': {
            'line': {'color': 'black', 'width': 5},
            'thickness': .75,
            'value': df['balance_value'].iloc[id]
        },
        
    }
))
text_value = df['life_balance'].iloc[id]
fig.add_annotation(
    x=0.5, y=0.1,  # Coordenadas en el gráfico (0-1)
    text=text_value,
    showarrow=False,
    font=dict(size=30)
)

fig.add_annotation(
    x=0.5, y=1.7,  # Coordenadas en el gráfico (0-1)
    text='Work Life Balance',
    showarrow=False,
    font=dict(size=30)
)

fig.update_layout(layout)

html_code = fig.to_html(full_html=False)

In [67]:
import plotly.io as pio

pio.fonts.findfont('Roboto')

AttributeError: module 'plotly.io' has no attribute 'fonts'